In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression


In [24]:
train_df = pd.read_csv("train_no_header.csv", header=None)
test_df = pd.read_csv("test_feature_no_header.csv", header=None)

#Giả sử cột cuối cùng trong train là nhãn (label)
X = train_df.iloc[:, :-1]
y = train_df.iloc[:, -1]
X_test = test_df.copy()

#2. Xác định cột số và cột phân loại 
num_cols = X.select_dtypes(include=['int64', 'float64']).columns
cat_cols = X.select_dtypes(include=['object']).columns

In [25]:
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),       # điền giá trị thiếu bằng trung bình
    ('scaler', StandardScaler())                       # chuẩn hoá
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),  # điền giá trị thiếu bằng mode
    ('encoder', OneHotEncoder(handle_unknown='ignore'))    # mã hoá one-hot
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

In [26]:
X_train_prepared = preprocessor.fit_transform(X)
X_test_prepared = preprocessor.transform(X_test)


In [27]:
log_reg = LogisticRegression(max_iter=1000, random_state=42)
scores = cross_val_score(log_reg, X_train_prepared, y, cv=5, scoring='f1')
print("F1 trung bình (cross-validation):", scores.mean())

log_reg.fit(X_train_prepared, y)
print(" Đã huấn luyện xong Logistic Regression!")

F1 trung bình (cross-validation): 0.8539949767604018
 Đã huấn luyện xong Logistic Regression!


In [28]:
y_pred = log_reg.predict(X_test_prepared)


In [32]:
submission = pd.DataFrame({
    'label': y_pred
})
submission.to_csv("0210768_NguyenTatManh_68CS2_submission.csv", index=False)
print(" Đã lưu kết quả vào file: 0210768_NguyenTatManh_68CS2_submission.csv")

 Đã lưu kết quả vào file: 0210768_NguyenTatManh_68CS2_submission.csv
